# Ensemble
`06_ensemble.ipynb`

- 정형 데이터 (Structured Data) 기준으로는 가장 뛰어난 상과를 내는 알고리즘 -> 앙상블 학습
- 대부분의 앙상블 학습 -> 트리 기반

## Random Forest
- 결정트리를 랜덤하게 만들어 트리의 숲을 만듦
- 각 결정트리의 예측을 종합해 최종 예측을 만듦
- 과대적합(overfitting)에 안전!

### 데이터 분할
- 데이터가 1000개면, 각 트리마다 1000개 데이터를 복원추출함. 이때 중복을 허용 (우연히 같은데이터만 1000개 가능)
- 노드 분할시, 분류 / 회귀의 특성 선택방식이 다름(분류: 개수를 루트함, 회귀: 특성을 다 씀)
- 기본값 100개의 트리를 만들어서
    - 분류: 다수결 투표
    - 회귀: 100개의 평균

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('./wine.csv')
X = wine[['alcohol', 'sugar', 'pH']]
y = wine['class']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)

# 교차검증(5 fold)
scores = cross_validate(rf, X_train, y_train, return_train_score=True, n_jobs=-1)

# 결정트리가 max_depth 튜닝이 없으면 Overfitting 됨 
#       훈련셋(일부)                        검증셋(훈련셋의 일부)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

# 진짜 학습 시키기
rf.fit(X_train, y_train)
# 트리(100개)들의 특성별 중요도 수치화

'''
이전 단일트리 특성 중요도: [0.12345626 0.86862934 0.0079144 ]
랜덤 포레스트 특성 중요도: [0.23167441 0.50039841 0.26792718]

노드마다 랜덤하게 특성들을 뽑아서 사용 -> 다양한 특성들이 훈련에서 기준으로 쓰임
-> 특정 특성에 과도하게 집중 방지 -> 다양한 특성이 훈련에 쓰일 기회를 받음 -> Overfitting을 줄이고 일반화 성능 높임
'''
print(rf.feature_importances_)

0.9973541965122431 0.8905151032797809
[0.23167441 0.50039841 0.26792718]


In [11]:
# OOB(Out Of Bag) 샘플. -> 쓰지않은 샘플 존재 -> 남은 데이터들을 모아서 만든 샘플 (훈련에 참여 안한 샘플)
# OOB샘플로 점수 -> 마치 검증샘플 같은 역할
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(X_train, y_train)
print(rf.score(X_train, y_train), rf.oob_score_)

0.996921300750433 0.8934000384837406


In [ ]:
# 랜덤 포레스트의 하이퍼 파라미터 튜닝

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint



